# Artificial Intelligence Project 2025/26  
## Class Timetable (CSP Problem)

---

### 1. Introduction
> This project aims to develop an intelligent agent capable of generating class timetables for the undergraduate courses at the Polytechnic Institute of Cávado and Ave (IPCA).  
> Timetable creation is a challenging and time-consuming task involving multiple constraints related to teachers, classrooms, and courses. Traditionally, this process requires significant manual effort and coordination.  
> 
> In this project, Artificial Intelligence techniques — specifically **Constraint Satisfaction Problems (CSP)** — are applied to automate the generation of valid and conflict-free timetables.  
> 
> The implementation is done in **Python**, using the **`python-constraint`** library, which provides a high-level API for defining variables, domains, and constraints in CSP problems.  
> The entire process is documented in a Jupyter Notebook as required by the Artificial Intelligence course guidelines.

**Group members:**
- Pedro Ribeiro — student number 27960  
- Ricardo Fernandes — student number 27961  
- Carolina Branco — student number 27983  
- João Barbosa — student number 27964  
- Diogo Abreu — student number 27975  

---

### 2. Goal Formulation

The main goal of this project is to **design a CSP-based intelligent agent** that automatically generates valid class timetables for IPCA’s undergraduate programs.

The agent must assign **courses to time slots, rooms, and teachers**, ensuring that no scheduling conflicts occur and that all academic and logistical restrictions are respected.

#### Limitations and Constraints
The timetable generation process is subject to multiple restrictions, including:

- **Teacher availability** — each teacher may have unavailable time slots.  
- **Room capacity and restrictions** — certain courses must take place in specific rooms (e.g., labs).  
- **Course-to-class assignments** — each class is linked to a specific set of courses.  
- **Online sessions** — some lessons occur online and do not require a physical room.  
- **Non-overlapping lessons** — a class, teacher, or room cannot be assigned to two lessons simultaneously.  
- **Lesson duration** — each session lasts 2 hours.  
- **Weekly limit** — each class has 10 lessons per week.  
- **Daily limit** — a class may have at most 3 lessons per day.  
- **Non-consecutive lessons** — lessons of the same course cannot be scheduled in consecutive slots.

#### Expected Results
The system should produce a **valid timetable** that:
- Assigns all courses to valid time slots and rooms.  
- Satisfies all **hard constraints**.  
- Avoids overlapping sessions and consecutive slots for the same class.  
- Handles both **in-person** and **online** sessions.  

---

### 3. Problem Formulation (CSP)

The problem was modeled as a **Constraint Satisfaction Problem (CSP)** using the `python-constraint` library.  
This library allows the definition of variables (with finite domains) and constraints (logical or functional) that the solver must satisfy simultaneously.

#### Variables
Each **lesson** of each **course** is represented by a group of variables:

- `<course>_<lesson>_slot` — the time slot assigned to that lesson (`1–20`).  
- `<course>_<lesson>_room` — the assigned classroom (`Lab01`, `Room1`, `Room2`, or `Online`).  
- `<course>_<lesson>_teacher` — the lecturer responsible for the course.  

#### Domains
- **Time slots:** integers from `1–20` (4 per day, across Monday–Friday).  
- **Rooms:** `["Lab01", "Room1", "Room2", "Online"]`, restricted when necessary.  
- **Teachers:** mapped automatically from the dataset.  

#### Constraints

**Hard Constraints (mandatory):**
- **Teacher availability:** lessons cannot be scheduled in teachers’ unavailable slots.  
- **Room restrictions:** some courses must occur in specific rooms (e.g., labs).  
- **Online classes:** lessons marked as online must be assigned to the "Online" room.  
- **No overlapping:**  
  - A class cannot have two lessons in the same time slot.  
  - A teacher cannot teach two classes at the same time.  
- **No consecutive lessons:** two lessons of the same class or in the same room cannot be consecutive.  
- **Daily limit:** no more than three lessons per day for any class.  

**Soft Constraints (optional / improvement goals):**
- Avoid scheduling multiple lessons of the same course on the same day.  
- Distribute lessons evenly throughout the week.  
- Minimize room changes when possible.  

---

### 4. Heuristics and Search

The `python-constraint` library internally uses backtracking with constraint propagation to find valid solutions.  
Although it does not provide explicit heuristic control (like MRV or LCV), the project indirectly improves performance by:

- Reducing variable domains where possible (e.g., applying room and teacher restrictions).  
- Structuring constraints pairwise to prune inconsistent assignments early.  

The solver searches for the **first valid solution** that satisfies all constraints.  
If multiple valid solutions exist, the first one found is exported for further analysis.

---

### 5. Solution Evaluation and Export

Once a valid timetable is found, it is stored and formatted as a structured **JSON** file (`outputs/best_schedule.json`).  

Each lesson is represented as:

```json
"UC11_1": {
    "slot": 5,
    "room": "Room2",
    "teacher": "jo"
}
```

The output JSON is sorted by course code for easier visualization.
Execution time is measured to evaluate solver efficiency.

### 6. Implementation Summary

The implementation pipeline can be summarized as follows:

**1. Dataset Import**

Reads input from `datasets/timetable_dataset.txt`, including:

- Class-to-course mappings

- Course-to-teacher assignments

- Time slot restrictions

- Room restrictions

- Online class flags

If no dataset is found, the script loads simulated data to ensure continuity.

**2. Variable Definition**

For each course and lesson, variables are created for:

- Slot (`1–20`)

- Room (restricted or general)

- Teacher (from dataset)

**3. Constraint Definition**

- Pairwise constraints to prevent consecutive lessons.

- `AllDifferentConstraint()` to avoid slot overlaps.

- Logical constraints for teacher unavailability and room usage.

- Custom function `max_three_lessons_per_day()` ensures the daily limit.

**4. Solving Process**

- The CSP is solved using `problem.getSolution()`.

- Execution time is logged for performance evaluation.

**5. Output Generation**

The resulting timetable is serialized into JSON format.

If writing fails, the solution is printed to the console as fallback.

### 7. Expected Results

The final system automatically generates valid, conflict-free timetables that:

- Respect all **hard constraints** (availability, room usage, no overlaps).

- Handle both **physical and online** classes.

- Prevent consecutive and overloaded days.

- Produce consistent JSON outputs ready for visualization or further optimization.

This demonstrates how **Constraint Satisfaction Programming** using the `python-constraint` library can effectively solve real-world scheduling problems with compact and readable Python code.

### 8. Repository Link
> 🔗 **GitHub Repository:** [https://github.com/diogooaabreu/IA25_P01_G4.git](https://github.com/diogooaabreu/IA25_P01_G4.git)

---


In [1]:
import json
from constraint import *
from collections import Counter 
import itertools
import time


# =========================================================================
# Definition of constants and constraint functions
# =========================================================================

rooms = ["Lab01", "Room1", "Room2", "Online"]
lessons_per_course = 2
slots = list(range(1, 21))

def no_consecutive_slots(*slots):
    """
    Ensures that no two lessons are scheduled in consecutive time slots.

    Args:
        *slots (int): A list of assigned time slot numbers (e.g., 1–20).

    Returns:
        bool: False if two lessons are in consecutive slots, True otherwise.
    """
    slots = sorted([s for s in slots if s is not None])
    for i in range(len(slots) - 1):
        if slots[i + 1] - slots[i] == 1:
            return False
    return True

def room_no_consecutive(room, *args):
    """
    Ensures that no two lessons in the same room are scheduled in consecutive time slots.

    Args:
        room (str): The room name to check.
        *args: A flattened sequence of alternating room and slot values 
               (e.g., room1, slot1, room2, slot2, ...).

    Returns:
        bool: False if two lessons in the same room are consecutive, True otherwise.
    """
    room_slot_pairs = [(args[i], args[i + 1]) for i in range(0, len(args), 2)]
    slots_in_room = sorted(slot for r, slot in room_slot_pairs if r == room and slot is not None)
    for i in range(len(slots_in_room) - 1):
        if slots_in_room[i + 1] - slots_in_room[i] == 1:
            return False
    return True

def get_day_from_slot(slot):
    """
    Determines the day of the week corresponding to a given time slot.

    Each day has 4 time slots:
        1–4   → Monday  
        5–8   → Tuesday  
        9–12  → Wednesday  
        13–16 → Thursday  
        17–20 → Friday  

    Args:
        slot (int): Time slot number (1–20).

    Returns:
        int: Day index (0 = Monday, 4 = Friday).
    """
    return (slot - 1) // 4

def max_three_lessons_per_day(*slots):
    """
    Ensures that no more than three lessons occur on the same day.

    Args:
        *slots (int): A list of assigned time slot numbers.

    Returns:
        bool: True if all days have three or fewer lessons, False otherwise.
    """
    slots = [s for s in slots if s is not None] 
    days = [get_day_from_slot(s) for s in slots]
    c = Counter(days)
    return all(v <= 3 for v in c.values())


# =============================
# 1. Import datasets
# =============================
dataset_path = "datasets/timetable_dataset.txt"

courses_assigned_to_classes = {}
courses_assigned_to_lecturers = {}
timeslot_restrictions = {}
roomrestrictions = {}
online_classes = {}
all_courses_with_variables = set() 

try:
    with open(dataset_path, "r", encoding="utf-8-sig") as f:
        lines = f.readlines()

    reading_section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue

        elif line.startswith("#cc"): reading_section = "cc"
        elif line.startswith("#dsd"): reading_section = "dsd"
        elif line.startswith("#tr"): reading_section = "tr"
        elif line.startswith("#rr"): reading_section = "rr"
        elif line.startswith("#oc"): reading_section = "oc"
        elif line.startswith("#"): reading_section = None
        else:
            if reading_section == "cc":
                parts = line.split()
                courses_assigned_to_classes[parts[0]] = parts[1:]
                for course in parts[1:]:
                     all_courses_with_variables.add(course)
            elif reading_section == "dsd":
                parts = line.split()
                courses_assigned_to_lecturers[parts[0]] = parts[1:]
            elif reading_section == "tr":
                parts = line.split()
                timeslot_restrictions[parts[0]] = list(map(int, parts[1:]))
            elif reading_section == "rr":
                parts = line.split()
                roomrestrictions[parts[0]] = parts[1:]
            elif reading_section == "oc":
                parts = line.split()
                online_classes[parts[0]] = list(map(int, parts[1:]))
                
except FileNotFoundError:
    print(f"ERROR: File not found at path: {dataset_path}")
    print("Using simulated data to continue the example.")
    courses_assigned_to_classes = {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15', 'UC21', 'UC22', 'UC23', 'UC24', 'UC25', 'UC31', 'UC32', 'UC33', 'UC34', 'UC35']}
    courses_assigned_to_lecturers = {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}
    timeslot_restrictions = {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}
    roomrestrictions = {'UC14': ['Lab01'], 'UC22': ['Lab01']}
    online_classes = {'UC21': [2], 'UC31': [2]}
    all_courses_with_variables = set(courses_assigned_to_classes['t01'])

# =============================
# 2. Create CSP and Constraints
# =============================

problem = Problem()

# 2.1 - Create variables
for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            slot_var = f"{course}_{lesson}_slot"
            room_var = f"{course}_{lesson}_room"
            teacher_var = f"{course}_{lesson}_teacher"
            
            problem.addVariable(slot_var, slots)
            
            if course in roomrestrictions:
                problem.addVariable(room_var, roomrestrictions[course])
            else:
                problem.addVariable(room_var, rooms)

            teachers_for_course = [t for t, c_list in courses_assigned_to_lecturers.items() if course in c_list]
            if teachers_for_course:
                problem.addVariable(teacher_var, teachers_for_course)

# 2.2 - No consecutive slots for the same class (pairwise)
for class_name, courses in courses_assigned_to_classes.items():
    slot_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    for i in range(len(slot_vars)):
        for j in range(i+1, len(slot_vars)):
            a = slot_vars[i]
            b = slot_vars[j]
            if a in problem._variables and b in problem._variables:
                problem.addConstraint(lambda s1, s2: abs(s1 - s2) != 1, (a, b))

# 2.3 - No consecutive classes in the same room (pairwise)
all_course_list = list(all_courses_with_variables)
for i in range(len(all_course_list)):
    for j in range(i+1, len(all_course_list)):
        c1 = all_course_list[i]
        c2 = all_course_list[j]
        for l1 in range(1, lessons_per_course + 1):
            for l2 in range(1, lessons_per_course + 1):
                room1 = f"{c1}_{l1}_room"
                slot1 = f"{c1}_{l1}_slot"
                room2 = f"{c2}_{l2}_room"
                slot2 = f"{c2}_{l2}_slot"
                if room1 in problem._variables and room2 in problem._variables and slot1 in problem._variables and slot2 in problem._variables:
                    def no_same_room_consec(r1, s1, r2, s2):
                        if r1 == r2:
                            return abs(s1 - s2) != 1
                        return True
                    problem.addConstraint(no_same_room_consec, (room1, slot1, room2, slot2))

# 2.4 - Teacher availability
for teacher, courses in courses_assigned_to_lecturers.items():
    unavailable_slots = timeslot_restrictions.get(teacher, [])
    for course in courses:
        if course in all_courses_with_variables: 
            for lesson in range(1, lessons_per_course + 1):
                slot_var = f"{course}_{lesson}_slot"
                if slot_var in problem._variables:
                    problem.addConstraint(
                        lambda s, unav=unavailable_slots: s not in unav,
                        (slot_var,)
                    )

# 2.5 - A class cannot have two lessons at the same slot
for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(AllDifferentConstraint(), lesson_vars)

# 2.6 - A teacher cannot teach two classes at the same time
for teacher, courses in courses_assigned_to_lecturers.items():
    lesson_vars = []
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            slot_var = f"{course}_{lesson}_slot"
            if slot_var in problem._variables:
                lesson_vars.append(slot_var)
    if lesson_vars:
        problem.addConstraint(AllDifferentConstraint(), lesson_vars)

# 2.7 - Online classes
for course, week_indexes in online_classes.items():
    if course in all_courses_with_variables: 
        for lesson in week_indexes:
            room_var = f"{course}_{lesson}_room"
            if room_var in problem._variables: 
                problem.addConstraint(
                    lambda r: r == "Online",
                    (room_var,)
                )

# 2.8 - Maximum of 3 lessons per day
for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(max_three_lessons_per_day, lesson_vars)


# =============================
# 3. Solve, Evaluate and Export JSON
# =============================

print("\n--- STARTING SEARCH (30s timeout) ---\n")

start_time = time.time()
solutions = []

def evaluate_solution(sol):
    """
    Evaluation function to score a solution based on soft constraints.
    Lower scores are better.
    
    Criteria:
    - Minimize number of room changes per teacher
    - (You can add more criteria later)
    """
    teacher_rooms = {}
    for var, val in sol.items():
        parts = var.split("_")
        course = parts[0]
        if len(parts) < 3:
            continue
        lesson_type = parts[2]
        if lesson_type == "room":
            lesson_index = parts[1]
            teacher_var = f"{course}_{lesson_index}_teacher"
            if teacher_var in sol:
                teacher = sol[teacher_var]
                room = val
                teacher_rooms.setdefault(teacher, []).append(room)

    total_changes = 0
    for teacher, rooms_used in teacher_rooms.items():
        for i in range(1, len(rooms_used)):
            if rooms_used[i] != rooms_used[i-1]:
                total_changes += 1

    return total_changes


# Custom generator to stop after 30s
for sol in problem.getSolutionIter():
    elapsed = time.time() - start_time
    if elapsed > 30:
        print(f"\nTime limit reached (30s). Stopping search.")
        break
    solutions.append(sol)

print(f"\nTotal solutions found within 30s: {len(solutions)}")

if not solutions:
    print("No valid solution found.")
else:
    # Evaluate all solutions and select the best one
    evaluated = [(evaluate_solution(sol), sol) for sol in solutions]
    best_score, best_solution = min(evaluated, key=lambda x: x[0])

    print(f"Best solution score: {best_score}")

    # Prepare JSON export
    schedule = {}
    for var, value in best_solution.items():
        parts = var.split('_')
        uc = parts[0]
        lesson_index = parts[1]
        var_type = parts[2]

        lesson_key = f"{uc}_{lesson_index}"
        if lesson_key not in schedule:
            schedule[lesson_key] = {}
        schedule[lesson_key][var_type] = value

    sorted_schedule = dict(sorted(schedule.items(), key=lambda x: x[0]))

    # Export to JSON
    output_filepath = r"outputs\best_schedule_pythonconstraint.json"
    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(sorted_schedule, f, indent=4)
        print(f"\nSuccessfully exported best solution to {output_filepath}")
    except Exception as e:
        print(f"ERROR writing file {output_filepath}: {e}")
        print(json.dumps(sorted_schedule, indent=4))

    # Show summary
    print("\n--- Example of Assignments (first 30 variables) ---")
    for i, var in enumerate(sorted(best_solution)):
        if i < 30:
            print(f"{var} → {best_solution[var]}")



--- STARTING SEARCH (30s timeout) ---


Time limit reached (30s). Stopping search.

Total solutions found within 30s: 8487
Best solution score: 13

Successfully exported best solution to outputs\best_schedule_pythonconstraint.json

--- Example of Assignments (first 30 variables) ---
UC11_1_room → Room2
UC11_1_slot → 4
UC11_1_teacher → jo
UC11_2_room → Online
UC11_2_slot → 1
UC11_2_teacher → jo
UC12_1_room → Online
UC12_1_slot → 12
UC12_1_teacher → mike
UC12_2_room → Online
UC12_2_slot → 10
UC12_2_teacher → mike
UC13_1_room → Online
UC13_1_slot → 16
UC13_1_teacher → rob
UC13_2_room → Online
UC13_2_slot → 14
UC13_2_teacher → rob
UC14_1_room → Lab01
UC14_1_slot → 20
UC14_1_teacher → rob
UC14_2_room → Lab01
UC14_2_slot → 18
UC14_2_teacher → rob
UC15_1_room → Online
UC15_1_slot → 8
UC15_1_teacher → sue
UC15_2_room → Online
UC15_2_slot → 6
UC15_2_teacher → sue


### 9. Critical Analysis and Future Improvements

#### 9.1 Critical Analysis

The obtained results demonstrate that the constraint-based model developed with the `python-constraint` library is effective in producing **valid and conflict-free timetables** that comply with all defined rules and restrictions.  
Nevertheless, several **limitations and improvement opportunities** were identified during testing and analysis:

- **Solution diversity** — While the solver can generate multiple valid solutions, many are structurally similar, differing only in minor time slot variations. This is due to the backtracking nature of the solver, which stops after finding the first valid configuration.  

- **Execution time** — For small to medium datasets, the system performs efficiently. However, the **search space increases exponentially** with additional classes, courses, and teachers, which can impact performance for larger datasets.  

- **Soft constraint handling** — The current implementation focuses primarily on hard constraints. There is no optimization function for soft preferences such as minimizing room changes or teacher idle times.  

- **Lack of heuristic customization** — Unlike solvers such as OR-Tools, `python-constraint` does not support advanced heuristics (e.g., MRV or LCV), which limits fine-tuned control over the search order.  

- **Practicality and realism** — Although all constraints are respected, some valid timetables may still be less practical (e.g., scattered lessons or excessive daily gaps).  

- **Solution exploration and evaluation** — An additional mechanism was implemented to explore multiple valid solutions within a 30-second time limit. Each solution is now evaluated using a custom scoring function, allowing comparison between alternative timetables and identification of the most balanced or optimal configuration. This enhances the system’s analytical capability beyond the first-found valid solution

#### 9.2 Future Improvements

To enhance the overall quality, scalability, and realism of the system, the following improvements are proposed:

1. **Multi-objective optimization**  
   Introduce custom scoring or ranking mechanisms to evaluate and choose between multiple valid solutions, considering soft constraints such as compactness, fairness, and balance.  

2. **Post-processing optimization**  
   Apply metaheuristic or local search techniques (e.g., simulated annealing, hill climbing, or genetic algorithms) to improve the first valid solution produced by the CSP solver.  

3. **Partial randomization and heuristics**  
   Implement randomized or heuristic-based variable selection to diversify search paths and avoid repetitive solution patterns.  

4. **Visualization and user interface**  
   Develop a timetable visualization tool (e.g., using `matplotlib` or a web interface) to display the generated schedules in a clear and intuitive manner.  

5. **Scalability and stress testing**  
   Test the system on larger datasets with multiple classes, teachers, and time slots to assess solver performance and identify optimization bottlenecks.  

---

### 10. Conclusion

This project successfully designed and implemented an **intelligent agent** capable of automatically generating valid academic timetables using **Constraint Satisfaction Problem (CSP)** techniques and the **`python-constraint`** library.  

Throughout the development process, the team gained valuable insights into **constraint modeling, problem decomposition, and solution validation** within a combinatorial scheduling context.

Additionally, the inclusion of a timed multi-solution search and evaluation mechanism allowed the system to assess and rank different timetable configurations within a fixed runtime limit, providing a foundation for future optimization-based extensions.

The agent effectively managed multiple **hard constraints**, including:
- Preventing overlaps between classes, teachers, and rooms;  
- Enforcing teacher availability restrictions;  
- Managing room assignments and lab requirements;  
- Handling online course scheduling;  
- Ensuring that no class exceeds three lessons per day;  
- Avoiding consecutive lessons within the same class or room.  

The implementation followed a modular and data-driven pipeline:
1. Dataset import and preprocessing.  
2. Definition of CSP variables and domains.  
3. Application of logical and functional constraints.  
4. Solving using `python-constraint`’s backtracking approach.  
5. Export of valid timetables to structured JSON format.  

The use of **Python** and the **`python-constraint`** library proved to be a simple yet effective approach for solving scheduling problems.  
While it lacks advanced heuristic control and optimization capabilities present in solvers like OR-Tools, it offers **high readability, rapid prototyping**, and easy constraint definition — ideal for educational and small-to-medium problem scales.

From a broader perspective, this project highlights the **practical applicability of Artificial Intelligence** to administrative and operational challenges such as academic scheduling.  
By leveraging constraint programming, the team was able to automate a traditionally manual process, reducing the potential for human error and improving timetable consistency.

Future work could explore:
- Incorporating **multi-objective optimization** for balancing soft constraints;  
- Enhancing the **user interface** for visualization and manual adjustment;  
- Integrating **metaheuristic methods** for hybrid optimization;  
- Conducting **large-scale performance tests** to assess solver scalability.  

In conclusion, the agent successfully achieved its goal of generating **valid, rule-compliant, and adaptable timetables**.  
The project provided hands-on experience in **AI-based constraint reasoning**, reinforcing the importance of structured modeling in solving complex scheduling and resource allocation problems.